In [1]:
# importing all libraries needed
import numpy as np
import pandas as pd
import os
import time
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

In [2]:
# saving the main directory
main_directory = os.getcwd()
main_directory

'C:\\Users\\greci\\Documents\\Jupiter_notebooks\\forDan'

In [3]:
#Rreading the name of the subfolder
subfolder_with_path = [f.path for f in os.scandir(main_directory) if f.is_dir()]

subfolder_with_path[-1]

'C:\\Users\\greci\\Documents\\Jupiter_notebooks\\forDan\\zzz_input_and_reference_files'

In [4]:
subfolder_name = [f.name for f in os.scandir(main_directory) if f.is_dir()]
subfolder_name

['.ipynb_checkpoints',
 '01Activity_files',
 '02Volume_files',
 '03Volume_in_tissue_files',
 'zzz_input_and_reference_files']

In [5]:
###############################################################################################
# Importing information from "samples overview", "intensisties template" and "brain allen map"
###############################################################################################

# Changing directory to the "zzz_input_and_reference_files" folder
os.chdir(subfolder_with_path[-1])


# Reading "sample_overview.csv" file
samples_overview = pd.read_csv("sample_overview.csv", 
                               dtype={"MOUSE": "string","SAMPLE": int,
                                      "FILES_HD": int,"FILES_BOX": int,
                                      "HEMI": "string","STAIN": "string",
                                      "TX_GROUP": "string", 
                                      "GENDER":"string", 
                                      "CONTEXT":"string"})
samples_overview

,MOUSE,SAMPLE,FILES_HD,FILES__BOX,hemisphere,marker,TX_GROUP,GENDER,CONTEXT
0,1001a,1,3778,3778,right,tdtomato,Saline,F,single housed
1,1001a,2,3632,3632,left,cfos,Saline,F,group housed
2,1002a,3,4074,4074,right,tdtomato,MDMA,F,single housed
3,1002a,4,3738,3738,left,cfos,MDMA,F,group housed
4,1003a,5,3854,3854,right,tdtomato,Saline,M,single housed
5,1003a,6,3870,3870,left,cfos,Saline,M,group housed
6,1004a,7,3820,3820,right,tdtomato,MDMA,M,single housed
7,1004a,8,3612,3612,left,cfos,MDMA,M,group housed
8,1005a,9,3840,3840,right,tdtomato,METH,F,single housed
9,1005a,10,4056,4056,left,cfos,METH,F,group housed


In [6]:
# Reading "intensities_template.csv" file
intensities_template = pd.read_csv("intensities_template.csv",
                                   dtype={"IDPath": "string",
                                          "LabelID": int, 
                                          "Region": "string", 
                                          "LabelAbrv": "string"})
intensities_template

,IDPath,LabelID,raw,LabelAbrv
0,/997/8/343/1129/1097/467/331/557/1/,1,tuberomammillary_nucleus_ventral_part,RTMv
1,/997/8/567/688/695/315/453/322/345/2/,2,primary_somatosensory_area_mouth_layer_6b,RSSp-m6b
2,/997/8/343/313/339/4/,4,inferior_colliculus,RIC
3,/997/1009/983/784/6/,6,internal_capsule,Rint
4,/997/8/343/1065/771/1132/7/,7,principal_sensory_nucleus_of_the_trigeminal,RPSV
...,...,...,...,...
551,/997/8/567/688/695/315/541/1127/,1127,temporal_association_areas_layer_2/3,RTEa2/3
552,/997/8/567/688/695/315/453/322/337/1128/,1128,primary_somatosensory_area_lower_limb_layer_5,RSSp-ll5
553,/997/8/567/688/695/698/619/1139/,1139,nucleus_of_the_lateral_olfactory_tract_layer_3,RNLOT3
554,/997/8/567/688/695/1089/1080/726/10703/,10703,"dentate_gyrus,_molecular_layer",RDG-mo


In [10]:
# Reading "brain_allen_map.csv" file
brain_allen_map = pd.read_csv("brain_allen_map.csv", 
                               dtype={"raw": "string","allen_1": "string",
                                      "allen_2": "string","allen_3": "string",
                                      "allen_4": "string","allen_5": "string",
                                      "allen_6": "string","allen_7": "string",
                                      "fine": "string","medium": "string",
                                      "coarse": "string", "all":"string"})
brain_allen_map

,raw,allen_1,allen_2,allen_3,allen_4,allen_5,allen_6,allen_7,fine,medium,coarse,all
0,endopiriform_nucleus_ventral_part,endopiriform_nucleus_ventral_part,endopiriform_nucleus,cortical_subplate,cortical_subplate,cortical_subplate,cerebral_cortex,cerebrum,endopiriform_nucleus,claustrum/endopiriform_nucleus,cerebral_cortex,all
1,endopiriform_nucleus_dorsal_part,endopiriform_nucleus_dorsal_part,endopiriform_nucleus,cortical_subplate,cortical_subplate,cortical_subplate,cerebral_cortex,cerebrum,endopiriform_nucleus,claustrum/endopiriform_nucleus,cerebral_cortex,all
2,posterior_amygdalar_nucleus,posterior_amygdalar_nucleus,posterior_amygdalar_nucleus,cortical_subplate,cortical_subplate,cortical_subplate,cerebral_cortex,cerebrum,posterior_amygdalar_nucleus,amygdalar_nuclei,amygdalar_nuclei,all
3,claustrum,claustrum,claustrum,cortical_subplate,cortical_subplate,cortical_subplate,cerebral_cortex,cerebrum,claustrum,amygdalar_nuclei,amygdalar_nuclei,all
4,basomedial_amygdalar_nucleus,basomedial_amygdalar_nucleus,basomedial_amygdalar_nucleus,cortical_subplate,cortical_subplate,cortical_subplate,cerebral_cortex,cerebrum,basomedial_amygdalar_nucleus,amygdalar_nuclei,amygdalar_nuclei,all
...,...,...,...,...,...,...,...,...,...,...,...,...
5545,doral_tegmental_decussation,other,other,other,other,other,other,other,other,other,other,all
5546,tectospil_pathway,other,other,other,other,other,other,other,other,other,other,all
5547,ventral_tegmental_decussation,other,other,other,other,other,other,other,other,other,other,all
5548,rubrospil_tract,other,other,other,other,other,other,other,other,other,other,all


In [11]:
# Changing directory to the "01Activity_files" folder
os.chdir(subfolder_with_path[1])

#Counting number of files on "01Activity_files" folder
list = os.listdir(subfolder_with_path[1]) 
number_files = len(list)

number_files

85

In [45]:
#Using and timing  a loop to process all files and populate the main data-frame

#Stanting timming process
start = time.time()

activity_data = [] #This array will contain df as elements

#Staring counters

i = 0 #General loop counter

sal_tdt = 0 # Counter of samples where Saline was used and tdTomato expressed
mdma_tdt = 0 # Counter of samples where MDMA was used and tdTomato expressed
meth_tdt = 0 # Counter of samples where Meth was used and tdTomato expressed

sal_cfos = 0 # Counter of samples where Saline was used and cfos expressed
mdma_cfos = 0 # Counter of samples where MDMA was used and cfos expressed
meth_cfos = 0 # Counter of samples where Meth was used and cfos expressed

#These array will contain sample's names as elements
samples_sal_tdt = [] 
samples_mdma_tdt = [] 
samples_meth_tdt = [] 
samples_sal_cfos = []
samples_mdma_cfos = [] 
samples_meth_cfos = []


    ##############################################
    # Innitiates loop to process each file
    ##############################################


while i < number_files:


    file = list[i]
    # Saving sample's number
    sample = file[5:7]
    
    # Saving rater's name
    rater = file[8:10]


    # Reading file's data
    regions_count_activity_fracc = pd.read_csv(file,low_memory=False)
    regions_count_activity_fracc = regions_count_activity_fracc[regions_count_activity_fracc.PIXEL_COUNT != "PIXEL_COUNT"]

    # Adding-up regions' fracctions
    regions_fracc_1 = regions_count_activity_fracc[["INTENSITY_1", "INTENSITY_1_PERC"]]
    regions_fracc_2 = regions_count_activity_fracc[["INTENSITY_2", "INTENSITY_2_PERC"]]
    regions_fracc_3 = regions_count_activity_fracc[["INTENSITY_3", "INTENSITY_3_PERC"]]
    regions_fracc_1 = regions_fracc_1.rename(columns={"INTENSITY_1": "INTENSITY","INTENSITY_1_PERC" : "COUNTS"})
    regions_fracc_2 = regions_fracc_2.rename(columns={"INTENSITY_2": "INTENSITY","INTENSITY_2_PERC" : "COUNTS"})
    regions_fracc_3 = regions_fracc_3.rename(columns={"INTENSITY_3": "INTENSITY","INTENSITY_3_PERC" : "COUNTS"})
    total_region_activity = regions_fracc_1.append(regions_fracc_2, ignore_index=True).append(regions_fracc_3, ignore_index=True)
    total_region_activity["COUNTS"] = total_region_activity["COUNTS"].astype(float)
    total_region_activity = total_region_activity.groupby(['INTENSITY']).agg('sum').reset_index()
    total_region_activity["COUNTS"] = total_region_activity["COUNTS"].round(0)
    total_region_activity["INTENSITY"] = total_region_activity["INTENSITY"].astype(int)

    # Adding activity columns to intensitites template data-frame
    output = intensities_template.merge(total_region_activity, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    activity_label_name = 'ACTIVITY'
    output = output.rename(columns = {'COUNTS':activity_label_name})

    # Adding a column of the sample number
    sample_column = np.empty(len(output))
    sample = int(sample)
    sample_column.fill(sample)
    output['SAMPLE'] = sample_column
    
    # Adding a column of the rater's name
    rater_column = np.empty(len(output))
    output['rater'] = rater_column
    output['rater'] = rater
    
    # Merging activity and sample overview data-frames
    output= pd.merge(output,samples_overview[['SAMPLE','GENDER',
                                             'hemisphere','marker',
                                             'TX_GROUP','CONTEXT']],on='SAMPLE',how='inner')
    
    #Adding a column with the sample's nickname

    if (output['TX_GROUP'].iloc[0] == 'Saline') and (output['marker'].iloc[0] == 'tdtomato'):
        if output['SAMPLE'].iloc[0] not in samples_sal_tdt:
            sal_tdt += 1
        samples_sal_tdt.append(output['SAMPLE'].iloc[0])
        nickname = 'SAL_' + str(sal_tdt)
        
    elif (output['TX_GROUP'].iloc[0] == 'MDMA') and (output['marker'].iloc[0] == 'tdtomato'):
        if output['SAMPLE'].iloc[0] not in samples_mdma_tdt:
            mdma_tdt += 1 
        samples_mdma_tdt.append(output['SAMPLE'].iloc[0])
        nickname = 'MDMA_' + str(mdma_tdt)
        
    elif (output['TX_GROUP'].iloc[0] == 'METH') and (output['marker'].iloc[0] == 'tdtomato'):
        if output['SAMPLE'].iloc[0] not in samples_meth_tdt:
            meth_tdt += 1 
        samples_meth_tdt.append(output['SAMPLE'].iloc[0])
        nickname = 'METH_' + str(meth_tdt)
        
    elif (output['TX_GROUP'].iloc[0] == 'Saline') and (output['marker'].iloc[0] == 'cfos'):
        if output['SAMPLE'].iloc[0] not in samples_sal_cfos:
            sal_cfos += 1 
        samples_sal_cfos.append(output['SAMPLE'].iloc[0])
        nickname = 'SAL_' + str(sal_cfos)
        
    elif (output['TX_GROUP'].iloc[0] == 'MDMA') and (output['marker'].iloc[0] == 'cfos'):
        if output['SAMPLE'].iloc[0] not in samples_mdma_cfos:
            mdma_cfos += 1 
        samples_mdma_cfos.append(output['SAMPLE'].iloc[0])
        nickname = 'MDMA_' + str(mdma_cfos)
        
    elif (output['TX_GROUP'].iloc[0] == 'METH') and (output['marker'].iloc[0] == 'cfos'):
        if output['SAMPLE'].iloc[0] not in samples_meth_cfos:
            meth_cfos += 1 
        samples_meth_cfos.append(output['SAMPLE'].iloc[0])
        nickname = 'METH_' + str(meth_cfos)

    nickname_column = np.empty(len(output))    
    output['nickname'] = nickname_column
    output['nickname'] = nickname

    
    ##########################################################################################################################
    
    # store DataFrame in list
    activity_data.append(output)
    
    i += 1
    
# see pd.concat documentation for more info
activity_data = pd.concat(activity_data)
end = time.time()


print(i,' Files processed ')
print(' ')
print(' Execution time:', round((end - start),2), 'seconds') 

# appended_data.to_excel('appended_data.xlsx',index=False)
activity_data

85  Files processed 
 
 Execution time: 35.12 seconds


,IDPath,LabelID,raw,LabelAbrv,ACTIVITY,SAMPLE,rater,GENDER,hemisphere,marker,TX_GROUP,CONTEXT,nickname
0,/997/8/343/1129/1097/467/331/557/1/,1,tuberomammillary_nucleus_ventral_part,RTMv,24.0,1.0,DR,F,right,tdtomato,Saline,single housed,SAL_1
1,/997/8/567/688/695/315/453/322/345/2/,2,primary_somatosensory_area_mouth_layer_6b,RSSp-m6b,33.0,1.0,DR,F,right,tdtomato,Saline,single housed,SAL_1
2,/997/8/343/313/339/4/,4,inferior_colliculus,RIC,4709.0,1.0,DR,F,right,tdtomato,Saline,single housed,SAL_1
3,/997/1009/983/784/6/,6,internal_capsule,Rint,315.0,1.0,DR,F,right,tdtomato,Saline,single housed,SAL_1
4,/997/8/343/1065/771/1132/7/,7,principal_sensory_nucleus_of_the_trigeminal,RPSV,234.0,1.0,DR,F,right,tdtomato,Saline,single housed,SAL_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
551,/997/8/567/688/695/315/541/1127/,1127,temporal_association_areas_layer_2/3,RTEa2/3,753.0,35.0,TH,M,right,tdtomato,MDMA,single housed,MDMA_7
552,/997/8/567/688/695/315/453/322/337/1128/,1128,primary_somatosensory_area_lower_limb_layer_5,RSSp-ll5,1435.0,35.0,TH,M,right,tdtomato,MDMA,single housed,MDMA_7
553,/997/8/567/688/695/698/619/1139/,1139,nucleus_of_the_lateral_olfactory_tract_layer_3,RNLOT3,411.0,35.0,TH,M,right,tdtomato,MDMA,single housed,MDMA_7
554,/997/8/567/688/695/1089/1080/726/10703/,10703,"dentate_gyrus,_molecular_layer",RDG-mo,1057.0,35.0,TH,M,right,tdtomato,MDMA,single housed,MDMA_7


In [36]:
activity_data.to_excel('activity_data.xlsx',index=False)

In [44]:

samples_sal_tdt = []
print(sal_tdt)
if output['SAMPLE'].iloc[0] not in samples_sal_tdt:
    sal_tdt += 1
print(sal_tdt)

30
31


In [19]:
pd.pivot_table(activity_data, values='ACTIVITY', index=['raw','hemisphere','marker','rater'],columns=['nickname'], aggfunc=np.sum)

nickname                                MDMA_1  MDMA_10  MDMA_11  MDMA_12  \
raw          hemisphere marker   rater                                      
Uvula_(iX)   right      tdtomato DR      177.0      NaN    100.0      NaN   
                                 JS        NaN      NaN      NaN     11.0   
                                 Or        NaN      NaN      NaN      NaN   
                                 PL        NaN      NaN      NaN      NaN   
                                 SS        NaN      NaN      NaN      NaN   
...                                        ...      ...      ...      ...   
zona_incerta right      tdtomato JS        NaN      NaN      NaN    449.0   
                                 Or        NaN      NaN      NaN      NaN   
                                 PL        NaN      NaN      NaN      NaN   
                                 SS        NaN      NaN      NaN      NaN   
                                 TH        NaN    584.0      NaN      NaN   

nickname                                MDMA_13  MDMA_14  MDMA_15  MDMA_16  \
raw          hemisphere marker   rater                                       
Uvula_(iX)   right      tdtomato DR         NaN      NaN      NaN     18.0   
                                 JS         NaN      NaN      NaN      NaN   
                                 Or         NaN      NaN      NaN      NaN   
                                 PL        40.0      NaN      NaN      NaN   
                                 SS         NaN      4.0      NaN      NaN   
...                                         ...      ...      ...      ...   
zona_incerta right      tdtomato JS         NaN      NaN      NaN      NaN   
                                 Or         NaN      NaN      NaN      NaN   
                                 PL       482.0      NaN      NaN      NaN   
                                 SS         NaN    374.0      NaN      NaN   
                                 TH         NaN      NaN    747.0      NaN   

nickname                                MDMA_17  MDMA_18  ...  SAL_28  SAL_29  \
raw          hemisphere marker   rater                    ...                   
Uvula_(iX)   right      tdtomato DR         NaN      NaN  ...     NaN     NaN   
                                 JS         6.0      NaN  ...     NaN     NaN   
                                 Or         NaN      NaN  ...     NaN     NaN   
                                 PL         NaN      6.0  ...   136.0     NaN   
                                 SS         NaN      NaN  ...     NaN    73.0   
...                                         ...      ...  ...     ...     ...   
zona_incerta right      tdtomato JS       368.0      NaN  ...     NaN     NaN   
                                 Or         NaN      NaN  ...     NaN     NaN   
                                 PL         NaN    502.0  ...   301.0     NaN   
                                 SS         NaN      NaN  ...     NaN   282.0   
                                 TH         NaN      NaN  ...     NaN     NaN   

nickname                                 SAL_3  SAL_30  SAL_4  SAL_5   SAL_6  \
raw          hemisphere marker   rater                                         
Uvula_(iX)   right      tdtomato DR        NaN     NaN    NaN    NaN  1006.0   
                                 JS        NaN     NaN    NaN    NaN     NaN   
                                 Or        NaN     NaN    NaN    NaN     NaN   
                                 PL     1033.0     NaN    NaN    NaN     NaN   
                                 SS        NaN     NaN  439.0    NaN     NaN   
...                                        ...     ...    ...    ...     ...   
zona_incerta right      tdtomato JS        NaN     NaN    NaN    NaN     NaN   
                                 Or        NaN     NaN    NaN    NaN     NaN   
                                 PL      391.0     NaN    NaN    NaN     NaN   
                                 SS        NaN   

In [ ]:
# Changing directory to the "02Volume_files" folder
os.chdir(subfolder_with_path[2])

#Counting number of files on "02Volume_files" folder
list = os.listdir(subfolder_with_path[2]) 
number_files = len(list)

number_files

In [ ]:
#Using and timing  a loop to process all files and populate the main data-frame


start = time.time()

volume_data = [] #This array will contain df as elements

i = 0
while i < number_files:

    ##############################################
    # Processing each file
    ##############################################
    
    file = list[i]
    # Saving sample's number 
    sample = file[5:7]


    # Reading file's data
    header = ["PIXEL_COUNT"]
    pixels_count_volume = pd.read_csv(file, names = header)
    pixels_count_volume.insert(0, column = 'INTENSITY', value = range(0, len(pixels_count_volume))) 

    # Adding activity columns to intensitites template data-frame
    output = intensities_template.merge(pixels_count_volume, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    volume_label_name = 'VOLUME'
    output = output.rename(columns = {'PIXEL_COUNT':volume_label_name})
    output['VOLUME'] = output['VOLUME'].div(1000000).round(6)

    # Adding a column of the sample number
    sample_column = np.empty(len(output))
    sample = int(sample)
    sample_column.fill(sample)
    output['SAMPLE'] = sample_column
    ##########################################################################################################################
    
    # store DataFrame in list
    volume_data.append(output)
    
    i += 1
    
#Joinning all elements of the array in a dataframe
volume_data = pd.concat(volume_data)
end = time.time()


print(i,' Files processed ')
print(' ')
print(' Execution time:', round((end - start),2), 'seconds') 

# appended_data.to_excel('appended_data.xlsx',index=False)
volume_data

In [ ]:
# Changing directory to the "03Volume_in_tissue_files" folder
os.chdir(subfolder_with_path[3])

#Counting number of files on "03Volume_in_tissue_files" folder
list = os.listdir(subfolder_with_path[3]) 
number_files = len(list)

number_files

In [ ]:
#Using and timing  a loop to process all files and populate the main data-frame


start = time.time()

volume_intissue_data = [] #This array will contain df as elements

i = 0
while i < number_files:

    ##############################################
    # Processing each file
    ##############################################
    
    file = list[i]
    # Saving sample's number 
    sample = file[5:7]


    # Reading file's data
    header = ["PIXEL_COUNT"]
    pixels_count_volume = pd.read_csv(file, names = header)
    pixels_count_volume.insert(0, column = 'INTENSITY', value = range(0, len(pixels_count_volume))) 

    # Adding activity columns to intensitites template data-frame
    output = intensities_template.merge(pixels_count_volume, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    volume_label_name = 'VOLUME_IN_TISSUE'
    output = output.rename(columns = {'PIXEL_COUNT':volume_label_name})
    output['VOLUME_IN_TISSUE'] = output['VOLUME_IN_TISSUE'].div(1000000).round(6)

    # Adding a column of the sample number
    sample_column = np.empty(len(output))
    sample = int(sample)
    sample_column.fill(sample)
    output['SAMPLE'] = sample_column
    ##########################################################################################################################
    
    # store DataFrame in list
    volume_intissue_data.append(output)
    
    i += 1
    
#Joinning all elements of the array in a dataframeo
volume_intissue_data = pd.concat(volume_intissue_data)
end = time.time()


print(i,' Files processed ')
print(' ')
print(' Execution time:', round((end - start),2), 'seconds') 

# appended_data.to_excel('appended_data.xlsx',index=False)
volume_intissue_data

In [ ]:
volumes_data = pd.merge(volume_intissue_data,volume_data[['VOLUME','SAMPLE']],on='SAMPLE',how='outer')
volumes_data['volumes_ratio'] = volumes_data['VOLUME_IN_TISSUE'] / volumes_data['VOLUME'] 
filtered_volumes_data = volumes_data.loc[volumes_data['volumes_ratio'] >= 0.5]
filtered_volumes_data = filtered_volumes_data.loc[filtered_volumes_data ['volumes_ratio'] <= 1.0]   
filtered_volumes_data

In [ ]:
test = pd.pivot_table(activity_data, values='ACTIVITY', index=['SAMPLE'],
                    columns=['LabelAbrv'], aggfunc=np.sum, dropna = True).reset_index('SAMPLE')


In [ ]:
all_data = pd.merge(activity_data,volume_data[['VOLUME']],on='SAMPLE',how='outer')

In [ ]:
all_data = pd.merge(activity_data,samples_overview[['SAMPLE','GENDER','TX_GROUP','CONTEXT']],on='SAMPLE',how='outer')
all_data = pd.merge(all_data,volume_data[['VOLUME','GENDER','TX_GROUP','CONTEXT']],on='SAMPLE',how='outer')
# all_data = all_data.dropna(subset=['LAAA'])
# Exporting dt to csv
# all_data.to_excel('all_data.xlsx',index=False)
all_data

In [ ]:
true_drug_label = samples_template[['SAMPLE', 'TX_GROUP']]


In [ ]:
number_of_rows = len(all_data) - 1

true_drug_label = samples_template[['SAMPLE', 'TX_GROUP']]
precited_drug_label = []

training_data = all_data[(all_data['SAMPLE'] != 1) | (all_data['SAMPLE'] != 3) | (all_data['SAMPLE'] != 9)]
y = training_data[["TX_GROUP"]]
x = training_data.drop(columns=["SAMPLE","TX_GROUP","CONTEXT","GENDER"])

test_data = all_data[(all_data['SAMPLE'] == 1) | (all_data['SAMPLE'] == 3) | (all_data['SAMPLE'] == 9)]
y_test = test_data[["TX_GROUP"]]
x_test = test_data.drop(columns=["SAMPLE","TX_GROUP","CONTEXT","GENDER"])


clf = LogisticRegression(solver='liblinear',multi_class='ovr').fit(x, y.values.ravel())
clf.fit(x,y.values.ravel())
prediction = clf.predict(x_test)
probabilities = clf.predict_proba(x_test)

    
#     output = i + 1, predictions[0],probabilities.item(0),probabilities.item(1),probabilities.item(2)


#     precited_drug_label.append(output)
    

# precited_drug_label
prediction
probabilities

In [ ]:
test_data.reset_index(drop=True)

In [ ]:
pd_pred = pd.DataFrame(prediction)
pd_prod = pd.DataFrame(probabilities)
test = pd_pred.merge(pd_prod, left_index=True, right_index=True)
test = test_data[["SAMPLE","TX_GROUP"]].reset_index(drop=True).merge(test, left_index=True, right_index=True)
test.columns =['SAMPLE', 'REAL_TX_GROUP', 'PREDICTED_TX_GROUP', 'PROB_MDMA', 'PROB_METH', 'SALINE']
test

In [ ]:
test.to_excel('test.xlsx',index=False)